In [ ]:
import numpy as np
import numba as nb
from numba import typed

from utils.utils import subset_2d, subset_2d_cached
from utils.bandit_utils import get_ci

In [ ]:
from algorithms.bandit_mips import action_elimination

# Constants
BATCH_SIZE = 50
HOEFFDING = 1.96  # Example constant for CI bound
SCALING_NUM_ATOMS = 5

# # Mock utility functions
# @nb.njit
# def subset_2d(array, rows, columns):
#     return array[rows][:, columns]

# @nb.njit
# def subset_2d_cached(source_array, rows, columns, use_cache, cache, cache_tracker, cache_map):
#     return source_array[rows][:, columns], len(rows) * len(columns), cache, cache_tracker, cache_map, 0

# def get_ci(delta, var_proxy, ci_bound, num_samples, pop_size, with_replacement):
#     return ci_bound * np.sqrt(var_proxy / num_samples)

# Synthetic data generation
def x(num_atoms=100, signal_dim=200, num_signals=10):
    atoms = np.random.rand(num_atoms, signal_dim)  # Atom features
    signals = np.random.rand(num_signals, signal_dim)  # Signals
    var_proxy = np.var(signals, axis=1)  # Variance proxy
    maxmin = np.array([np.max(signals), np.min(signals)])  # Max and min for scaling
    return atoms, signals, var_proxy, maxmin

# Generate dataset
atoms, signals, var_proxy, maxmin = generate_synthetic_data()

# Run the algorithm
candidates_array, budgets_array, cache, cache_tracker, cache_map = action_elimination(
    atoms=atoms,
    signals=signals,
    var_proxy=var_proxy,
    maxmin=maxmin,
    epsilon=0.1,
    delta=0.05,
    var_proxy_override=False,
    num_best_atoms=2,
    abs=False,
    verbose=True,
    with_replacement=False,
    batch_size=BATCH_SIZE,
    seed=42,
    use_cache=False,
    permutation=np.arange(atoms.shape[1]),
    cache=np.empty((atoms.shape[0], atoms.shape[1])),
    cache_tracker=np.zeros(atoms.shape[0], dtype=np.int64),
    cache_map=typed.List([typed.Dict.empty(key_type=nb.int64, value_type=nb.int64) for _ in range(atoms.shape[0])]),
)

print("Selected Candidates:", candidates_array)
print("Budgets Used:", budgets_array)
